### Import Statements

In [1]:
import geopandas as gpd

import cuxfilter
from cuxfilter.layouts import feature_and_five_edge, double_feature
import cudf
import numpy as np

import holoviews as hv
import pandas as pd

from pyproj import Proj, Transformer



### Importing HouseHold Distance Data

In [2]:
df_harris = pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/distances_charleston_county.parquet')
df_harris = df_harris[df_harris['DIST'] != 4]
df_harris['AGE'] = np.where(((df_harris['head_hh_age_code'] == 'J') | (df_harris['head_hh_age_code'] == 'K') | 
                            (df_harris['head_hh_age_code'] == 'L') | (df_harris['head_hh_age_code'] == 'M')), 1, 2)
df_harris.rename(columns = {'children_ind': 'CHILD'}, inplace = True)
df_harris

,CHILD,head_hh_age_code,GE_LONGITUDE_2010,GE_LATITUDE_2010,tank_lat,tank_lon,ERQK_RISKS,SWND_RISKS,HRCN_RISKS,TRND_RISKS,CFLD_RISKS,RFLD_RISKS,AVG_RISK,dist_mi,DIST,AGE
0,0,C,-79.9308,32.7765,32.761407,-79.950218,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,2
1,0,M,-79.9308,32.7765,32.761407,-79.950218,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,1
2,0,L,-79.9308,32.7765,32.761407,-79.950218,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,1
3,1,D,-79.9308,32.7765,32.761407,-79.950218,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,2
4,0,M,-79.9308,32.7765,32.761407,-79.950218,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245095,0,A,-79.9917,32.7870,32.826088,-79.962751,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,3.181200,3,2
245096,0,F,-79.9917,32.7870,32.826088,-79.962751,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,3.181200,3,2
245097,0,I,-79.9917,32.7870,32.826088,-79.962751,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,3.181200,3,2
245098,0,B,-79.9917,32.7870,32.826088,-79.962751,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,3.181200,3,2


### Reading in Harris County Tanks

In [3]:
df_tanks_harris = gpd.read_file('/hpc/group/codeplus22-vis/infousa_copy/charleston_tanks.shp')
df_tanks_harris.head(n=3)

,object_cla,nw_corner_,nw_corne_1,se_corner_,se_corne_1,geometry
0,sedimentation_tank,32.830983,-79.945656,32.830585,-79.945212,"POLYGON ((-79.94521 32.83098, -79.94521 32.830..."
1,sedimentation_tank,32.830853,-79.946247,32.830470,-79.945797,"POLYGON ((-79.94580 32.83085, -79.94580 32.830..."
2,closed_roof_tank,32.825241,-79.944935,32.824854,-79.944696,"POLYGON ((-79.94470 32.82524, -79.94470 32.824..."


### Finding average of tank coordinates

In [4]:
df_tanks_harris['tank_lat'] = (df_tanks_harris['nw_corner_'] + df_tanks_harris['se_corner_'])/2
df_tanks_harris['tank_lon'] = (df_tanks_harris['nw_corne_1'] + df_tanks_harris['se_corne_1'])/2
df_tanks_harris = df_tanks_harris[['object_cla', 'tank_lat', 'tank_lon']]
df_tanks_harris

,object_cla,tank_lat,tank_lon
0,sedimentation_tank,32.830784,-79.945434
1,sedimentation_tank,32.830662,-79.946022
2,closed_roof_tank,32.825047,-79.944816
3,closed_roof_tank,32.825263,-79.943791
4,closed_roof_tank,32.826536,-79.941263
...,...,...,...
245,spherical_tank,32.827541,-79.939549
246,spherical_tank,32.827387,-79.940151
247,closed_roof_tank,32.825002,-79.939474
248,closed_roof_tank,32.824928,-79.939648


### Transforming Latitude/Longitude Coordinates

In [5]:
transform_4326_to_3857 = Transformer.from_crs('epsg:4326', 'epsg:3857')
df_tanks_harris['LATITUDE_TX'], df_tanks_harris['LONGITUDE_TX'] = transform_4326_to_3857.transform(
                                                df_tanks_harris['tank_lat'], df_tanks_harris['tank_lon']
                                            )

df_tanks_harris = df_tanks_harris.drop(['tank_lat', 'tank_lon'], axis=1)
df_tanks_harris

,object_cla,LATITUDE_TX,LONGITUDE_TX
0,sedimentation_tank,-8.899485e+06,3.872865e+06
1,sedimentation_tank,-8.899550e+06,3.872849e+06
2,closed_roof_tank,-8.899416e+06,3.872105e+06
3,closed_roof_tank,-8.899302e+06,3.872134e+06
4,closed_roof_tank,-8.899021e+06,3.872302e+06
...,...,...,...
245,spherical_tank,-8.898830e+06,3.872435e+06
246,spherical_tank,-8.898897e+06,3.872415e+06
247,closed_roof_tank,-8.898822e+06,3.872099e+06
248,closed_roof_tank,-8.898841e+06,3.872089e+06


In [6]:
df_tanks_harris['is_tank'] = 1
df_tanks_harris

,object_cla,LATITUDE_TX,LONGITUDE_TX,is_tank
0,sedimentation_tank,-8.899485e+06,3.872865e+06,1
1,sedimentation_tank,-8.899550e+06,3.872849e+06,1
2,closed_roof_tank,-8.899416e+06,3.872105e+06,1
3,closed_roof_tank,-8.899302e+06,3.872134e+06,1
4,closed_roof_tank,-8.899021e+06,3.872302e+06,1
...,...,...,...,...
245,spherical_tank,-8.898830e+06,3.872435e+06,1
246,spherical_tank,-8.898897e+06,3.872415e+06,1
247,closed_roof_tank,-8.898822e+06,3.872099e+06,1
248,closed_roof_tank,-8.898841e+06,3.872089e+06,1


In [7]:
# Apply transformation
transform_4326_to_3857 = Transformer.from_crs('epsg:4326', 'epsg:3857')
df_harris['LATITUDE_TX'], df_harris['LONGITUDE_TX'] = transform_4326_to_3857.transform(
                                                df_harris['GE_LATITUDE_2010'], df_harris['GE_LONGITUDE_2010']
                                            )

df_harris = df_harris.drop(['GE_LATITUDE_2010', 'GE_LONGITUDE_2010', 'tank_lat', 'tank_lon'], axis=1)
df_harris

,CHILD,head_hh_age_code,ERQK_RISKS,SWND_RISKS,HRCN_RISKS,TRND_RISKS,CFLD_RISKS,RFLD_RISKS,AVG_RISK,dist_mi,DIST,AGE,LATITUDE_TX,LONGITUDE_TX
0,0,C,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,2,-8.897856e+06,3.865676e+06
1,0,M,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,1,-8.897856e+06,3.865676e+06
2,0,L,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,1,-8.897856e+06,3.865676e+06
3,1,D,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,2,-8.897856e+06,3.865676e+06
4,0,M,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,1,-8.897856e+06,3.865676e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245095,0,A,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,3.181200,3,2,-8.904635e+06,3.867066e+06
245096,0,F,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,3.181200,3,2,-8.904635e+06,3.867066e+06
245097,0,I,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,3.181200,3,2,-8.904635e+06,3.867066e+06
245098,0,B,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,3.181200,3,2,-8.904635e+06,3.867066e+06


In [8]:
df_harris

,CHILD,head_hh_age_code,ERQK_RISKS,SWND_RISKS,HRCN_RISKS,TRND_RISKS,CFLD_RISKS,RFLD_RISKS,AVG_RISK,dist_mi,DIST,AGE,LATITUDE_TX,LONGITUDE_TX
0,0,C,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,2,-8.897856e+06,3.865676e+06
1,0,M,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,1,-8.897856e+06,3.865676e+06
2,0,L,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,1,-8.897856e+06,3.865676e+06
3,1,D,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,2,-8.897856e+06,3.865676e+06
4,0,M,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3,1,-8.897856e+06,3.865676e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245095,0,A,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,3.181200,3,2,-8.904635e+06,3.867066e+06
245096,0,F,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,3.181200,3,2,-8.904635e+06,3.867066e+06
245097,0,I,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,3.181200,3,2,-8.904635e+06,3.867066e+06
245098,0,B,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,3.181200,3,2,-8.904635e+06,3.867066e+06


In [9]:
df_tanks_harris

,object_cla,LATITUDE_TX,LONGITUDE_TX,is_tank
0,sedimentation_tank,-8.899485e+06,3.872865e+06,1
1,sedimentation_tank,-8.899550e+06,3.872849e+06,1
2,closed_roof_tank,-8.899416e+06,3.872105e+06,1
3,closed_roof_tank,-8.899302e+06,3.872134e+06,1
4,closed_roof_tank,-8.899021e+06,3.872302e+06,1
...,...,...,...,...
245,spherical_tank,-8.898830e+06,3.872435e+06,1
246,spherical_tank,-8.898897e+06,3.872415e+06,1
247,closed_roof_tank,-8.898822e+06,3.872099e+06,1
248,closed_roof_tank,-8.898841e+06,3.872089e+06,1


### Merging Household, Distance, Tank Data

In [10]:
df_harris = df_harris.append(df_tanks_harris, ignore_index = True)
df_harris

,CHILD,head_hh_age_code,ERQK_RISKS,SWND_RISKS,HRCN_RISKS,TRND_RISKS,CFLD_RISKS,RFLD_RISKS,AVG_RISK,dist_mi,DIST,AGE,LATITUDE_TX,LONGITUDE_TX,object_cla,is_tank
0,0.0,C,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3.0,2.0,-8.897856e+06,3.865676e+06,NaN,NaN
1,0.0,M,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3.0,1.0,-8.897856e+06,3.865676e+06,NaN,NaN
2,0.0,L,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3.0,1.0,-8.897856e+06,3.865676e+06,NaN,NaN
3,1.0,D,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3.0,2.0,-8.897856e+06,3.865676e+06,NaN,NaN
4,0.0,M,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3.0,1.0,-8.897856e+06,3.865676e+06,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.898830e+06,3.872435e+06,spherical_tank,1.0
146294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.898897e+06,3.872415e+06,spherical_tank,1.0
146295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.898822e+06,3.872099e+06,closed_roof_tank,1.0
146296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.898841e+06,3.872089e+06,closed_roof_tank,1.0


### Multiselect

In [11]:
df_harris['is_tank'] = df_harris['is_tank'].fillna(0)

df_harris['DIST'] = df_harris['DIST'].fillna(0)

df_harris['dist_mi'] = df_harris['dist_mi'].fillna(6)

df_harris['AGE'] = df_harris['AGE'].fillna(0)
df_harris['CHILD'] =df_harris['CHILD'].fillna(3.0)
df_harris

,CHILD,head_hh_age_code,ERQK_RISKS,SWND_RISKS,HRCN_RISKS,TRND_RISKS,CFLD_RISKS,RFLD_RISKS,AVG_RISK,dist_mi,DIST,AGE,LATITUDE_TX,LONGITUDE_TX,object_cla,is_tank
0,0.0,C,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3.0,2.0,-8.897856e+06,3.865676e+06,NaN,0.0
1,0.0,M,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3.0,1.0,-8.897856e+06,3.865676e+06,NaN,0.0
2,0.0,L,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3.0,1.0,-8.897856e+06,3.865676e+06,NaN,0.0
3,1.0,D,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3.0,2.0,-8.897856e+06,3.865676e+06,NaN,0.0
4,0.0,M,23.668781,12.914367,10.610779,18.06919,26.632081,10.998755,17.148992,1.536279,3.0,1.0,-8.897856e+06,3.865676e+06,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146293,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,0.0,0.0,-8.898830e+06,3.872435e+06,spherical_tank,1.0
146294,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,0.0,0.0,-8.898897e+06,3.872415e+06,spherical_tank,1.0
146295,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,0.0,0.0,-8.898822e+06,3.872099e+06,closed_roof_tank,1.0
146296,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,0.0,0.0,-8.898841e+06,3.872089e+06,closed_roof_tank,1.0


In [12]:
label_map_dist = {0: 'Tank', 1: '0.5 miles away', 
             2: '1 mile away', 3: '5 miles away'}

label_map_age = {0: 'Tank', 1: 'Elderly', 
             2: 'Not Elderly'}

label_map_children = {0: 'No Children', 1: 'Children', 
             2: 'Tank'}

label_map_tank = {0: 'No Tanks', 1: 'Tank'}

colors = ['#05c1ff', '#ff0000', '#ff00a4', '#a11aeb']




### Transforming to cuxfilter dataframe

In [13]:
cdf = cudf.DataFrame.from_pandas(df_harris) 

In [14]:
cux_df = cuxfilter.DataFrame.from_dataframe(cdf) 

### Making Cuxfilter Charts

In [15]:
harris = cuxfilter.charts.scatter(x='LATITUDE_TX', y='LONGITUDE_TX', pixel_shade_type='linear', color_palette = colors, aggregate_fn = 'max', aggregate_col = 'DIST', tile_provider="CartoDark", title = 'Households in Charleston County in Close Proximity to Tanks',
                                   x_range=(-13825798.514061378,-7542228.134036879), y_range=(2819963.842141629,6272600.009501693), legend = True)

# charleston = cuxfilter.charts.scatter(x='LATITUDE_SC', y='LONGITUDE_SC', pixel_shade_type='linear', color_palette = colors, aggregate_fn = 'max', aggregate_col = 'DIST', tile_provider="CartoDark", title = 'Households in Charleston County in Close Proximity to Tanks',
#                                    x_range=(-13825798.514061378,-7542228.134036879), y_range=(2819963.842141629,6272600.009501693), legend = True)

dist = cuxfilter.charts.multi_select('DIST', label_map=label_map_dist)

age = cuxfilter.charts.multi_select('AGE', label_map=label_map_age)

children = cuxfilter.charts.multi_select('CHILD', label_map=label_map_children)

dist_slider = cuxfilter.charts.range_slider('dist_mi')

tank = cuxfilter.charts.drop_down('is_tank', label_map = label_map_tank)

charts_list = [harris, dist, age, children, dist_slider]

In [16]:
# d = cux_df.dashboard(charts_list, layout = feature_and_five_edge)

# d = cux_df.dashboard([harris], sidebar = [dist, age, children, dist_slider], layout = cuxfilter.layouts.feature_and_triple_base, theme = cuxfilter.themes.rapids) 

d = cux_df.dashboard([harris, dist_slider, tank], sidebar = [dist, age, children], layout = cuxfilter.layouts.feature_and_double_base, theme = cuxfilter.themes.rapids) 


### Dashboard

In [17]:
d.show()
d.app(sidebar_width=200) # run the dashboard within the notebook cell

Dashboard running at port 34489


Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_height', width=200)
        [0] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='Datapoints Selected')
            [0] Column(sizing_mode='stretch_both')
                [0] Number(css_classes=['indicator'], font_size='18pt', format='{value:,}', sizing_mode='stretch_width', value=146298)
                [1] Progress(sizing_mode='stretch_width', value=100)
        [1] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='DIST_multi_select', width=400)
            [0] MultiSelect(height=200, options={'Tank': 0, '0.5 miles awa...}, sizing_mode='stretch_both', value=[''], width=400)
        [2] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='AGE_multi_select', width=400)
            [0] MultiSelect(height=200, options={'Tank': 0, 'Elderly': 1, ...}, sizing_mode='stretch_both', value=[''], width=400)
        [3] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='CHILD_multi_select', width=400)
            [0] MultiSelect(height=200, options={'No Children': 0, ...}, sizing_mode='stretch_both', value=[''], width=400)
    [1] GridStack(allow_drag=False, allow_resize=False, ncols=11, nrows=6, sizing_mode='stretch_both')
        [0] Card(sizing_mode='stretch_both', title='Households in C..., width=800)
            [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')
        [1] Card(sizing_mode='stretch_both', title='dist_mi_range_slider', width=400)
            [0] RangeSlider(end=6.0, sizing_mode='stretch_both', start=0.013255532840467104, value=(0.013255532840467104, ..., value_end=6.0, value_start=0.013255532840467104)
        [2] Card(sizing_mode='stretch_both', title='is_tank_dropdown', width=400)
            [0] Select(height=50, options={'No Tanks': 0, ...}, sizing_mode='fixed', width=400)